In [1]:
import pandas as pd

# CSVファイルを読み込む
df = pd.read_csv("emdat.csv")

# データの先頭5行を表示
print(df.head())

# カラム名を確認
print("CSVのカラム一覧:", df.columns)

  Disaster Subgroup      Disaster Type  Disaster Subtype  ISO  \
0    Meteorological              Storm  Tropical cyclone  USA   
1      Hydrological              Flood   Flood (General)  JAM   
2        Biological           Epidemic     Viral disease  JAM   
3       Geophysical  Volcanic activity          Ash fall  JPN   
4       Geophysical         Earthquake   Ground movement  TUR   

                    Country                        Subregion    Region  \
0  United States of America                 Northern America  Americas   
1                   Jamaica  Latin America and the Caribbean  Americas   
2                   Jamaica  Latin America and the Caribbean  Americas   
3                     Japan                     Eastern Asia      Asia   
4                   Türkiye                     Western Asia      Asia   

   Latitude  Longitude  Start Year  Total Deaths  
0       NaN        NaN        1900        6000.0  
1       NaN        NaN        1900         300.0  
2       NaN

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# CSVファイル読み込み
df = pd.read_csv("emdat.csv")

# カラム名確認
print("CSVのカラム一覧:", df.columns)

# 列名を柔軟に対応
def find_column(possible_names):
    for name in df.columns:
        if name.lower() in [p.lower() for p in possible_names]:
            return name
    return None

year_col = find_column(["Year", "year", "YEAR"])
deaths_col = find_column(["Deaths", "deaths", "DEATHS"])

# 年代別死者数グラフ
if year_col and deaths_col:
    deaths_by_year = df.groupby(year_col)[deaths_col].sum()

    plt.figure(figsize=(10,6))
    deaths_by_year.plot(kind="bar", color="red")
    plt.title("年代別 死者数")
    plt.xlabel("Year")
    plt.ylabel("Number of Deaths")
    plt.tight_layout()
    plt.show()
else:
     print("⚠️ 'Year' または 'Deaths' 列が見つかりません。")

CSVのカラム一覧: Index(['Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'ISO',
       'Country', 'Subregion', 'Region', 'Latitude', 'Longitude', 'Start Year',
       'Total Deaths'],
      dtype='object')
⚠️ 'Year' または 'Deaths' 列が見つかりません。


In [3]:
import pandas as pd
import folium

# CSVファイル読み込み
df = pd.read_csv("emdat.csv")

# カラム名確認（緯度・経度の列名をチェック）
print("CSVのカラム一覧:", df.columns)

# 緯度・経度の列名を指定（例: Latitude, Longitude）
lat_col = "Latitude"
lon_col = "Longitude"

# 地図の中心をデータの平均値で設定
map_center = [df[lat_col].mean(), df[lon_col].mean()]
m = folium.Map(location=map_center, zoom_start=4)

print("✅ マップのインプット準備ができました。次にマーカーを追加して可視化できます。")

CSVのカラム一覧: Index(['Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'ISO',
       'Country', 'Subregion', 'Region', 'Latitude', 'Longitude', 'Start Year',
       'Total Deaths'],
      dtype='object')
✅ マップのインプット準備ができました。次にマーカーを追加して可視化できます。


In [4]:
# 必要なライブラリ
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# --- 0. df の安全確認＆準備 ---
def ensure_dataframe(obj):
    """obj が DataFrame でなければ DataFrame に変換、存在しなければエンプティを返す"""
    if obj is None:
        return pd.DataFrame()
    if isinstance(obj, pd.DataFrame):
        return obj
    try:
        return pd.DataFrame(obj)
    except Exception:
        return pd.DataFrame()

# 既存の df を安全に取得（未定義なら None）
df = globals().get("df", None)
df = ensure_dataframe(df)

# columns の安全表示（print でコケないように）
cols = list(df.columns) if hasattr(df, "columns") else []
print("Before:", cols)

# --- 1. 列名リネーム（存在する列だけ） ---
rename_map = {
    "災害名": "disaster_name",
    "場所": "location",
    "緯度": "lat",
    "経度": "lon",
    "日付": "date",
    # 必要に応じて追加
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

print("After:", df.columns.tolist())

# --- 2. データ整形 ---
# 日付の正規化
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 緯度・経度の型と欠損処理
for c in ("lat", "lon"):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if {"lat", "lon"}.issubset(df.columns):
    df = df.dropna(subset=["lat", "lon"])

# データが空なら早期終了（地図だけは出す）
if df.empty or not {"lat", "lon"}.issubset(df.columns):
    m = folium.Map(location=[35.68, 139.76], zoom_start=5)
    m.save("disaster_map.html")
    print("データが不足または空のため、ベースマップのみを保存しました: disaster_map.html")
else:
    # --- 3. マップ作成 ---
    m = folium.Map(location=[35.68, 139.76], zoom_start=5)
    marker_cluster = MarkerCluster().add_to(m)

    # --- 4. マーカー追加 ---
    for _, row in df.iterrows():
        # ポップアップ内容を安全に作成
        name = row["disaster_name"] if "disaster_name" in df.columns else ""
        loc = row["location"] if "location" in df.columns else ""
        date_str = ""
        if "date" in df.columns and pd.notna(row["date"]):
            try:
                date_str = pd.to_datetime(row["date"]).strftime("%Y-%m-%d")
            except Exception:
                date_str = str(row["date"])

        popup_html = f"""
        <b>災害名:</b> {name}<br>
        <b>場所:</b> {loc}<br>
        <b>日付:</b> {date_str}
        """

        folium.Marker(
            location=[float(row["lat"]), float(row["lon"])],
            popup=popup_html,
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(marker_cluster)

    # --- 5. 保存 ---
    m.save("disaster_map.html")
    print("地図を保存しました: disaster_map.html")


Before: ['Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'ISO', 'Country', 'Subregion', 'Region', 'Latitude', 'Longitude', 'Start Year', 'Total Deaths']
After: ['Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'ISO', 'Country', 'Subregion', 'Region', 'Latitude', 'Longitude', 'Start Year', 'Total Deaths']
データが不足または空のため、ベースマップのみを保存しました: disaster_map.html
